<a href="https://colab.research.google.com/github/navneetkrc/Geo_data_science/blob/master/ReverseGeocoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on this [Blog](https://towardsdatascience.com/reverse-geocoding-in-python-a915acf29eb6). 

In [0]:
# Install GDAL and Geopandas
!apt install gdal-bin python-gdal python3-gdal --quiet
!apt install python3-rtree --quiet
!pip install git+git://github.com/geopandas/geopandas.git --quiet
!pip install descartes --quiet
!pip install geopy
!pip install plotly_express
!pip install ipython-autotime


* Restart the run time. tqdm does not work with recent python so we downgrade it to get it working

In [0]:
!pip install tqdm==4.36.0

In [0]:
%load_ext autotime
import pandas as pd

import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

## Reverse Geocoding Single Example

In [2]:
locator = Nominatim(user_agent="myGeocoder")
coordinates = "53.480837, -2.244914"
location = locator.reverse(coordinates)

time: 69 ms


In [3]:
location.raw

{'address': {'building': 'Eagle Insurance Buildings',
  'city': 'Manchester',
  'country': 'United Kingdom',
  'country_code': 'gb',
  'county': 'Greater Manchester',
  'house_number': '68',
  'postcode': 'M2 4JG',
  'road': 'Cross Street',
  'state': 'England',
  'state_district': 'North West England',
  'suburb': 'City Centre'},
 'boundingbox': ['53.480856', '53.4810634', '-2.2451761', '-2.2449576'],
 'display_name': 'Eagle Insurance Buildings, 68, Cross Street, City Centre, Manchester, Greater Manchester, North West England, England, M2 4JG, United Kingdom',
 'lat': '53.4809597',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'lon': '-2.2450668274629235',
 'osm_id': 37139875,
 'osm_type': 'way',
 'place_id': 96393663}

time: 9.51 ms


In [4]:
print(location.address)

Eagle Insurance Buildings, 68, Cross Street, City Centre, Manchester, Greater Manchester, North West England, England, M2 4JG, United Kingdom
time: 3 ms


In [5]:
location.raw["display_name"]

'Eagle Insurance Buildings, 68, Cross Street, City Centre, Manchester, Greater Manchester, North West England, England, M2 4JG, United Kingdom'

time: 4.11 ms


## Reverse Geocoding with Pandas

In [6]:
url = "https://www.dropbox.com/s/15gisj8hx218rn1/street-pole-sample.csv?dl=1"
cols = ["X", "Y", "POLE_NUM","TYPE","HEIGHT","POLE_DATE","OWNER"]
df = pd.read_csv(url, usecols=cols)
df.head()

,X,Y,POLE_NUM,TYPE,HEIGHT,POLE_DATE,OWNER
0,-75.170097,39.942766,214423,WP,NaN,1997-06-09T00:00:00.000Z,PECO
1,-75.166112,39.941477,215645,AAPT,25.0,1997-06-10T00:00:00.000Z,Streets
2,-75.163483,39.943068,215926,WP,NaN,1997-06-04T00:00:00.000Z,PECO
3,-75.167727,39.944528,214515,WP,25.0,1997-06-09T00:00:00.000Z,Streets
4,-75.171863,39.941486,214380,WP,25.0,1997-06-09T00:00:00.000Z,Streets


time: 1.11 s


In [7]:
df.shape

(150, 7)

time: 3.43 ms


In [8]:
import plotly_express as px
px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")
px.scatter_mapbox(df, lat="Y", lon="X",  zoom=15)

time: 663 ms


In [9]:
df["geom"] =  df["Y"].map(str)  + ',' + df['X'].map(str)
df.head()

,X,Y,POLE_NUM,TYPE,HEIGHT,POLE_DATE,OWNER,geom
0,-75.170097,39.942766,214423,WP,NaN,1997-06-09T00:00:00.000Z,PECO,"39.9427660880249,-75.17009743393821"
1,-75.166112,39.941477,215645,AAPT,25.0,1997-06-10T00:00:00.000Z,Streets,"39.941477314134396,-75.166112027818"
2,-75.163483,39.943068,215926,WP,NaN,1997-06-04T00:00:00.000Z,PECO,"39.9430681055253,-75.1634826347411"
3,-75.167727,39.944528,214515,WP,25.0,1997-06-09T00:00:00.000Z,Streets,"39.9445279530865,-75.16772651013079"
4,-75.171863,39.941486,214380,WP,25.0,1997-06-09T00:00:00.000Z,Streets,"39.9414861490123,-75.17186304554191"


time: 30.6 ms


In [10]:
df.geom[0]



'39.9427660880249,-75.17009743393821'

time: 3.18 ms


In [11]:
locator = Nominatim(user_agent="myGeocoder", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)
rgeocode

time: 6.54 ms


In [0]:
tqdm.pandas()
df['address'] = df['geom'].progress_apply(rgeocode)

In [13]:
df.head()


,X,Y,POLE_NUM,TYPE,HEIGHT,POLE_DATE,OWNER,geom,address
0,-75.170097,39.942766,214423,WP,NaN,1997-06-09T00:00:00.000Z,PECO,"39.9427660880249,-75.17009743393821","(723, South Chadwick Street, Hawthorne, South ..."
1,-75.166112,39.941477,215645,AAPT,25.0,1997-06-10T00:00:00.000Z,Streets,"39.941477314134396,-75.166112027818","(Tindley Temple United Methodist Church, 750, ..."
2,-75.163483,39.943068,215926,WP,NaN,1997-06-04T00:00:00.000Z,PECO,"39.9430681055253,-75.1634826347411","(600, South 13th Street, Hawthorne, South Phil..."
3,-75.167727,39.944528,214515,WP,25.0,1997-06-09T00:00:00.000Z,Streets,"39.9445279530865,-75.16772651013079","(1538, Naudain Street, Rittenhouse Square, Phi..."
4,-75.171863,39.941486,214380,WP,25.0,1997-06-09T00:00:00.000Z,Streets,"39.9414861490123,-75.17186304554191","(Marian Anderson Community Center, Catharine S..."


time: 28.3 ms


In [14]:
df["address"][0]

Location(723, South Chadwick Street, Hawthorne, South Philadelphia, Philadelphia, Philadelphia County, Pennsylvania, 19146, United States of America, (39.94273438888889, -75.17000927777778, 0.0))

time: 4.93 ms
